<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/RAG-AVA-Mistral-7B/blob/main/RAG_AVA_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<p align="center" width="100%">
  <img src="https://github.com/mehdihosseinimoghadam/AVA-Mistral-7B/blob/main/ava3.png?raw=true" alt="Sublime's custom image"/>
</p>




In [ ]:
!pip install sentence-transformers metapub
!pip install accelerate
!pip install transformers accelerate bitsandbytes -q
! pip install transformers trl py7zr optimum accelerate bitsandbytes
! pip install git+https://github.com/huggingface/peft
! pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.3/120.3 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 30.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 84.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 104.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 55.3 MB/s eta 0:00:00
  Created wheel for m

In [ ]:
# Rstart your session now

In [ ]:
from sentence_transformers import SentenceTransformer, util
import os
import csv
import time
import torch
import pandas as pd

In [ ]:
paper_text = pd.read_csv('bio_data.csv').dropna().reset_index()['Abstract']
paper_text

0     دلیل و هدف: اختلالات گلومرولی سیر بالینی بسیار...
1     گیرنده های سیگنال مصنوعی، پاسخ های سلولی قابل ...
2     استفاده از هوش مصنوعی (AI) در کشف دارو منجر به...
3     اهداف: سویا به دلیل داشتن پروتئین و روغن بالا،...
4     کلکالیزاسیون پاتوژن های میکروبی و پپتید β-آمیل...
5     پتانسیل آنزیم های مهندسی شده در کاربردهای صنعت...
6     بسیاری از پروتئین ها در یک ساختار سه بعدی ثابت...
7     ابریشم، به ویژه ابریشم عنکبوت و حشرات، یک ماده...
8     با بیش از 270 رخداد منحصر به فرد در ژنوم انسان...
9     کارسینوم یوروتلیال پیشرفته یا متاستاتیک محلی (...
10    در دهه‌های گذشته، تاشو TIM-barrel به‌عنوان یک ...
11    قابلیت‌های منحصر به فرد و دقیق پروتئین‌ها به د...
12    کربنیک آنهیدرازها (CAs) به دلیل سادگی محل فعال...
13    اطلاعات موجود در پروتئین‌ها از دنباله‌ای به سا...
14    آنزیم‌ها زیست‌کاتالیست‌های ضروری برای کاربردها...
15    اثربخشی تجویز دوزهای بالای ویتامین D به بیمارا...
16    چندین ماده مغذی در تقویت سیستم ایمنی و حفظ یکپ...
17    پیش زمینه: اشتراکات ساختاری و عملکردی بین 

In [ ]:


if not torch.cuda.is_available():
  print("Warning: No GPU detected. Processing will be slow. Please add a GPU to this notebook")

model1 = SentenceTransformer('LABSE')


print("Encode the corpus. This might take a while")
corpus_embeddings = model1.encode(paper_text, show_progress_bar=True, convert_to_tensor=True)



###############################
print("Corpus loaded with {} sentences / embeddings".format(len(paper_text)))

Encode the corpus. This might take a while


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Corpus loaded with 57 sentences / embeddings


In [ ]:
# Function that searches the corpus and prints the results
def search(inp_question, num_res):
    res = {}
    start_time = time.time()
    question_embedding = model1.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings)
    end_time = time.time()
    hits = hits[0]  #Get the hits for the first query

    print("Input question:", inp_question)
    print("Results (after {:.3f} seconds):".format(end_time-start_time))
    for hit in hits[0:num_res]:

        res[hit['corpus_id']] = paper_text[hit['corpus_id']]
    df = pd.DataFrame(list(res.items()),columns = ['id','res'])
    return df

In [ ]:
search("طراحی پروتئین De novo چیست؟", 5)

Input question: طراحی پروتئین De novo چیست؟
Results (after 0.116 seconds):


,id,res
0,6,بسیاری از پروتئین ها در یک ساختار سه بعدی ثابت...
1,13,اطلاعات موجود در پروتئین‌ها از دنباله‌ای به سا...
2,35,چشم انداز بیوتکنولوژیک شاهد رشد قابل توجهی در ...
3,52,تجمع پروتئین، در نتیجه تا کردن اشتباه و اختلال...
4,38,خلوص بالای DNA پلاسمید (pDNA)، به‌ویژه در ایزو...


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

In [ ]:
model_name_or_id = "MehdiHosseiniMoghadam/AVA-Mistral-7B-V2"
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" # the device to load the model onto

model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
q = "طراحی پروتئین De novo چیست؟"

res = search(q, 4)


prompt = f'''

با توجه به شرایط زیر به این سوال پاسخ دهید:

{q},

متن نوشته: {res['res'][0]} - {res['res'][1]} - {res['res'][2]} - {res['res'][3]}

'''


prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.99,
    max_new_tokens=900,
    pad_token_id=tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Input question: طراحی پروتئین De novo چیست؟
Results (after 0.019 seconds):
### Human:

با توجه به شرایط زیر به این سوال پاسخ دهید:

طراحی پروتئین De novo چیست؟,

متن نوشته: بسیاری از پروتئین ها در یک ساختار سه بعدی ثابت جمع نمی شوند، بلکه در حالت بسیار بی نظم عمل می کنند. این پروتئین‌های ذاتاً بی‌نظم چالشی منحصربه‌فرد برای مهندسی و طراحی پروتئین ایجاد می‌کنند: چگونه می‌توان پروتئین‌ها را به‌صورت نو طراحی کرد، اگر نه با طراحی ساختارشان؟ در اینجا، زمینه نوپای طراحی پروتئین‌های دارای اختلال ذاتی را با تمرکز بر کاربردها در بیوتکنولوژی و پزشکی مرور خواهیم کرد. اهداف طراحی لزوماً نباید مانند طراحی de novo پروتئین های تا شده باشد، زیرا پروتئین های بی نظم دارای نقاط قوت و محدودیت های عملکردی منحصر به فردی هستند. ما بر روی عملکردهایی تمرکز می‌کنیم که در آن پروتئین‌های ذاتاً بی‌نظم به طور منحصربه‌فردی مناسب هستند، از جمله پیوندهای نامنظم، چاپرون‌های خشک‌کننده، حسگرهای محیط شیمیایی، تحویل داروها، و ترکیبات میعانات بیومولکولی. طراحی پروتئین‌های دارای اختلال ذاتی عملکردی متکی بر ترکیبی از ابزارهای 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
q = 'چاپرون های مولکولی در تاخوردگی پروتئین چیست؟'

res = search(q, 4)


prompt = f'''

با توجه به شرایط زیر به این سوال پاسخ دهید:

{q},

متن نوشته: {res['res'][0]} - {res['res'][1]} - {res['res'][2]} - {res['res'][3]}

'''


prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.99,
    max_new_tokens=900,
    pad_token_id=tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Input question: چاپرون های مولکولی در تاخوردگی پروتئین چیست؟
Results (after 0.020 seconds):
### Human:

با توجه به شرایط زیر به این سوال پاسخ دهید:

چاپرون های مولکولی در تاخوردگی پروتئین چیست؟,

متن نوشته: پروتئین ها ماشین های مولکولی هستند که ساختار را فراهم می کنند و نقش های مهم انتقال، سیگنال و آنزیمی را انجام می دهند. پروتئین های بیان شده توسط سلول ها نیاز به تنظیم دقیق غلظت، تا شدن، محلی سازی و تغییرات دارند. با این حال، این حالت هموستاز پروتئین به طور مداوم توسط تنش های سطح بافت مختل می شود. در حالی که سلول‌های بافت‌های سالم می‌توانند در برابر این اختلالات بافر شوند، به عنوان مثال، با بیان پروتئین‌های چپرون، هموستاز پروتئین در پیری از بین می‌رود و می‌تواند منجر به تجمع پروتئین مشخصه بیماری‌های تاخوردگی پروتئین شود. در اینجا، گزارش‌هایی از قطع ارتباط تدریجی بین تنظیم ترانسکریپتومی و پروتئومی در طول پیری سلولی را مرور می‌کنیم. ما در مورد اینکه چگونه تغییرات مرتبط با سن در پاسخ‌های سلولی به عوامل استرس‌زای خاص در ریزمحیط بافتی با از دست دادن پروتئین‌های ریبوزومی، مکث ریبوزومی و ترج

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
q = 'کدام کشور تنوع ژنتیکی غنی برای پرورش سویا دارد ؟'

res = search(q, 4)


prompt = f'''

با توجه به شرایط زیر به این سوال پاسخ دهید:

{q},

متن نوشته: {res['res'][0]} - {res['res'][1]} - {res['res'][2]} - {res['res'][3]}

'''


prompt = f"### Human:{prompt}\n### Assistant:"


inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.99,
    max_new_tokens=900,
    pad_token_id=tokenizer.eos_token_id
)


outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Input question: کدام کشور تنوع ژنتیکی غنی برای پرورش سویا دارد ؟
Results (after 0.020 seconds):
### Human:

با توجه به شرایط زیر به این سوال پاسخ دهید:

کدام کشور تنوع ژنتیکی غنی برای پرورش سویا دارد ؟,

متن نوشته: اهداف: سویا به دلیل داشتن پروتئین و روغن بالا، یک محصول خوراکی و روغنی مهم در جهان است. چین مجموعه ای از بیش از 43000 منبع ژرم پلاسم سویا دارد که تنوع ژنتیکی غنی برای پرورش سویا فراهم می کند. با این حال، تنوع ژنتیکی غنی، چالش های بزرگی را برای بهبود ژنتیکی سویا ایجاد می کند. این مطالعه در مورد مجموعه ژنوم de novo HJ117، یک رقم سویا با محتوای پروتئین بالا 52.99٪ گزارش می دهد. این داده‌ها منابع ارزشمندی برای تحقیقات بیشتر در بهبود کیفیت دانه سویا هستند و به روشن شدن مکانیسم‌های تنظیمی زیربنای محتوای پروتئین سویا کمک خواهند کرد.
توصیف داده ها: ما یک ژنوم مرجع پیوسته 1041.94 مگابایت برای HJ117 با استفاده از ترکیبی از خواندن کوتاه Illumina (23.38 گیگابیت) و خواندن طولانی PacBio (25.58 گیگابایت)، با پوشش توالی با کیفیت بالا تقریباً 22.44×5×22.44×، 2، ایجاد کردیم. HJ117 از طریق اصل